# Setup

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.io import arff
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Loading the data

In [2]:
#Get route of the path
current_path = os.getcwd()
aux_curr_path = current_path
project_path = aux_curr_path.replace('/notebooks', '')
dataset_path = "dataset/CEE_DATA.arff"
dataset_path = os.path.join(project_path, dataset_path)

data, meta = arff.loadarff(dataset_path)

df = pd.DataFrame(data)
df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x) #Encoding from byte to string 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Performance           666 non-null    object
 1   Gender                666 non-null    object
 2   Caste                 666 non-null    object
 3   coaching              666 non-null    object
 4   time                  666 non-null    object
 5   Class_ten_education   666 non-null    object
 6   twelve_education      666 non-null    object
 7   medium                666 non-null    object
 8   Class_ X_Percentage   666 non-null    object
 9   Class_XII_Percentage  666 non-null    object
 10  Father_occupation     666 non-null    object
 11  Mother_occupation     666 non-null    object
dtypes: object(12)
memory usage: 62.6+ KB


            #0 : Average -  157
            #1 : Excellent - 101
            #2 : Good - 210
            #3 : Very Good - 198

# Preprocessing and Feature Engineering

In [3]:
columns_of_interest=["Performance",'Class_ X_Percentage', 'Class_XII_Percentage', 'medium', 'Caste']
updated_df=df[columns_of_interest]

In [4]:
y= updated_df[['Performance']]

In [5]:
#Create oneHot enconder object
enc_OneHot = OneHotEncoder(sparse_output=False)

#Applying OneHot
y_OneHot = enc_OneHot.fit_transform(y)

#Create Label encoder object
ord_enc=LabelEncoder()

#Applying LabelEnconder to y
df["y_ord_enc"]=ord_enc.fit_transform(y)
y_Label = df["y_ord_enc"]


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
updated_df = updated_df.assign(y_coding_col=y_Label.values)
updated_df

,Performance,Class_ X_Percentage,Class_XII_Percentage,medium,Caste,y_coding_col
0,Excellent,Excellent,Excellent,ENGLISH,General,1
1,Excellent,Excellent,Excellent,OTHERS,OBC,1
2,Excellent,Excellent,Excellent,ENGLISH,OBC,1
3,Excellent,Excellent,Excellent,OTHERS,General,1
4,Excellent,Excellent,Excellent,ENGLISH,General,1
...,...,...,...,...,...,...
661,Average,Good,Vg,ENGLISH,ST,0
662,Average,Vg,Good,ENGLISH,ST,0
663,Average,Good,Vg,ENGLISH,ST,0
664,Average,Good,Good,ENGLISH,ST,0


In [7]:
df_av_class_0 = updated_df[updated_df["y_coding_col"]==0]
df_ex_class_1 = updated_df[updated_df["y_coding_col"]==1]
df_gd_class_2 = updated_df[updated_df["y_coding_col"]==2]
df_vg_class_3 = updated_df[updated_df["y_coding_col"]==3]

In [8]:
updated_df.y_coding_col.value_counts()

y_coding_col
2    210
3    198
0    157
1    101
Name: count, dtype: int64

--------

In [9]:
Number_of_samples = 180

#Oversampling
df_av_class_0_over =  df_av_class_0.sample(Number_of_samples, replace=True)
df_ex_class_1_over =  df_ex_class_1.sample(Number_of_samples, replace=True)

#Undersampling
df_gd_class_2_under = df_gd_class_2.sample(Number_of_samples)
df_vg_class_3_under = df_vg_class_3.sample(Number_of_samples)

df_mod_samples = pd.concat([df_av_class_0_over,
                            df_ex_class_1_over,
                            df_gd_class_2_under,
                            df_vg_class_3_under],axis=0) 

df_mod_samples.Performance.value_counts()

Performance
Average      180
Excellent    180
Good         180
Vg           180
Name: count, dtype: int64

In [10]:
df_mod_samples.head()

,Performance,Class_ X_Percentage,Class_XII_Percentage,medium,Caste,y_coding_col
578,Average,Vg,Vg,ENGLISH,ST,0
605,Average,Excellent,Vg,ENGLISH,ST,0
537,Average,Excellent,Excellent,ENGLISH,SC,0
503,Average,Excellent,Excellent,ASSAMESE,OBC,0
552,Average,Vg,Vg,ENGLISH,SC,0


------

In [11]:
df_mod_samples.columns

Index(['Performance', 'Class_ X_Percentage', 'Class_XII_Percentage', 'medium',
       'Caste', 'y_coding_col'],
      dtype='object')

In [12]:
cols2drop = ["Performance", 'y_coding_col']
X = df_mod_samples.drop(cols2drop, axis=1)


y = df_mod_samples[["Performance"]]

In [13]:
X

,Class_ X_Percentage,Class_XII_Percentage,medium,Caste
578,Vg,Vg,ENGLISH,ST
605,Excellent,Vg,ENGLISH,ST
537,Excellent,Excellent,ENGLISH,SC
503,Excellent,Excellent,ASSAMESE,OBC
552,Vg,Vg,ENGLISH,SC
...,...,...,...,...
136,Excellent,Excellent,OTHERS,General
189,Good,Vg,ENGLISH,OBC
250,Excellent,Excellent,ENGLISH,General
173,Excellent,Vg,OTHERS,OBC


In [14]:
y

,Performance
578,Average
605,Average
537,Average
503,Average
552,Average
...,...
136,Vg
189,Vg
250,Vg
173,Vg


In [15]:
#Columns to apply one hot enconder
col_X=['Class_ X_Percentage', 'Class_XII_Percentage', 'medium', 'Caste']

#Create the transformer
ct= ColumnTransformer(
    transformers=[
    ("OneHotInXColumns", enc_OneHot,col_X)
                      ]
)

In [17]:
# Applying OneHot to X
X = ct.fit_transform(X)

In [18]:
X

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [19]:
y

,Performance
578,Average
605,Average
537,Average
503,Average
552,Average
...,...
136,Vg
189,Vg
250,Vg
173,Vg


In [20]:
y_Label

0      1
1      1
2      1
3      1
4      1
      ..
661    0
662    0
663    0
664    0
665    0
Name: y_ord_enc, Length: 666, dtype: int64

In [21]:
#Applying OneHot
y_OneHot = enc_OneHot.fit_transform(y)

#Get Label enconder
y_Label = df_mod_samples["y_coding_col"]

In [22]:
y_OneHot

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [23]:
y_Label

578    0
605    0
537    0
503    0
552    0
      ..
136    3
189    3
250    3
173    3
252    3
Name: y_coding_col, Length: 720, dtype: int64

-------

# Splitting the dataset

In [24]:
#Split the dataset for y_OneHot
X_train, X_test, y_train_OneHot, y_test_OneHot = train_test_split(X, y_OneHot, test_size=0.2, random_state=42)

#Split the dataset for y_Label (Pandas Series)
X_train, X_test, y_train_Label, y_test_Label = train_test_split(X, y_Label, test_size=0.2, random_state=42)



In [37]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train_OneHot)
y_pred = clf.predict(X_test)



print(classification_report(y_test_OneHot, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82        40
           1       0.50      0.07      0.12        42
           2       0.22      0.06      0.10        31
           3       1.00      0.10      0.18        31

   micro avg       0.71      0.28      0.40       144
   macro avg       0.64      0.26      0.31       144
weighted avg       0.64      0.28      0.32       144
 samples avg       0.28      0.28      0.28       144



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:

Experiments = [
    (
        "Random Forest n_estimators=100", 
        RandomForestClassifier(class_weight="balanced"),
        (X_train, y_train_OneHot),
        (X_test, y_test_OneHot)
    ),

    (
        "XGBoost",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train, y_train_OneHot),
        (X_test, y_test_OneHot)
    ),
    
    (
         "Multinomial Logistic Regression",
        LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200),
        (X_train, y_train_Label),
        (X_test, y_test_Label)
    ),
    
     (
        "K-Nearest Neighbors",
        KNeighborsClassifier(n_neighbors=5),
        (X_train, y_train_Label),
        (X_test, y_test_Label)
    ),
    
    (
        "MLP",
        MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, activation='relu', solver='adam'),
        (X_train, y_train_OneHot),
        (X_test, y_test_OneHot)
    ),
    
    (
        "Support Vector Classifier",
        SVC(kernel='linear', probability=True),  
        (X_train, y_train_Label),
        (X_test, y_test_Label)
    )
    
    
]


In [27]:
results_per_model = []

for model_name, model, train_set, test_set in Experiments:
    X_train = train_set[0] #get Xtrain from the list models
    y_train = train_set[1] #get y_train from list models
    X_test = test_set[0]    #get x_test from list models 
    y_test = test_set[1]   #get y_test from list models
    
    model.fit(X_train, y_train)  #train the current model
    y_pred = model.predict(X_test) #make predictions 
    report = classification_report(y_test, y_pred, output_dict=True) #make a dict of the classification report
    
    
    results_per_model.append(report) #add the previus dict to a list

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [16:32:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic

In [38]:
results_per_model

[{'0': {'precision': 0.8421052631578947,
   'recall': 0.8,
   'f1-score': 0.8205128205128205,
   'support': 40},
  '1': {'precision': 0.5714285714285714,
   'recall': 0.09523809523809523,
   'f1-score': 0.16326530612244897,
   'support': 42},
  '2': {'precision': 0.25,
   'recall': 0.06451612903225806,
   'f1-score': 0.10256410256410256,
   'support': 31},
  '3': {'precision': 0.625,
   'recall': 0.16129032258064516,
   'f1-score': 0.2564102564102564,
   'support': 31},
  'micro avg': {'precision': 0.7049180327868853,
   'recall': 0.2986111111111111,
   'f1-score': 0.41951219512195126,
   'support': 144},
  'macro avg': {'precision': 0.5721334586466165,
   'recall': 0.28026113671274966,
   'f1-score': 0.3356881214024071,
   'support': 144},
  'weighted avg': {'precision': 0.588952850877193,
   'recall': 0.2986111111111111,
   'f1-score': 0.3528184778184778,
   'support': 144},
  'samples avg': {'precision': 0.2986111111111111,
   'recall': 0.2986111111111111,
   'f1-score': 0.298611111

In [29]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train_OneHot)
y_pred = clf.predict(X_test)



print(classification_report(y_test_OneHot, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82        40
           1       0.50      0.07      0.12        42
           2       0.36      0.13      0.19        31
           3       1.00      0.10      0.18        31

   micro avg       0.72      0.29      0.42       144
   macro avg       0.68      0.27      0.33       144
weighted avg       0.67      0.29      0.34       144
 samples avg       0.29      0.29      0.29       144



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
